# Music 128 Final Project: Sonifying Colors
### Giuseppe Perona
## I) Introduction:
This document involves running code in a Jupyter Notebook (this document). Click once on this cell and you should see it become selected if it isn't already. Then, press `shift` and `return` if you're using a Mac and `alt` and `enter` if you're using Windows. This will execute this cell (which will do nothing, since this is a text cell), and move you to the next cell. The next cell imports all of the packages we'll need for this project, such as `numpy`, a powerful tool for generating and storing lists of numbers. We're also using `matplotlib` to plot graphs, and `simpleaudio` to play sounds. If you find that the audio doesn't work, please use the mybinder.org link that should be included with the submission. If you accidentally edit the code, try reopening the mybinder.org link. If your edit remains, send me an email, and I should be able to revert to the previous email easily. If mybinder.org isn't working, send me an email, and I will try to fix it. It is important to run each cell in order unless I state otherwise. There are some points that you can jump back to, such as the beginning of each quiz question that will be pointed out. Now, execute the next cell with all the `import` statements.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
try:
    import simpleaudio as sa
except:
    %pip install simpleaudio
    import simpleaudio as sa
from IPython.display import Audio
import ipywidgets as widgets
from IPython.display import display
from mpl_toolkits.mplot3d import Axes3D


## II) Setup
In the next cell, we have some code that forms the backbone of our sonification. In this project I am representing colors as sounds. A color has a red, a green, and a blue value, each of which ranges from 0 to 255. In my sonification, each color will be a mixture of three notes: A, C# and E. These notes will be played at volumes corresponding to its red, green, and blue values, with A corresponding to red, C# corresponding to green, and E corresponding to green. Therefore, a deep purple $(50,0,50)$ will be the A and the E played at low volume, simultaneously. A bright orange $(255, 100, 0)$ will be the A played loudly and the C# played less loudly. In this project, the A is 440 Hz, the C# is the next highest C# (554 Hz), and the E is the next highest E (650 Hz). To play the sounds, we sample the sine waves with our desired parameters for A, C# and E, 44100 times per second, and then add them together. Feel free to read the comments in green about the code, but there is no need to understand them or the code itself. Now, run the next cell (`shift` + `return` or equivalent).

In [ ]:
# calculate note frequencies
A_freq = 440
Csh_freq = A_freq * 2 ** (4 / 12)
E_freq = A_freq * 2 ** (7 / 12)

c = 255/np.log(255)

# get timesteps for each sample, T is note duration in seconds
sample_rate = 44100
T = 1
interval = int(sample_rate*T)
t = np.linspace(0, T, interval, False)

# generate sine wave notes
def generate_sound(rgb):
    sin_scal = t * 2 * np.pi

    def generate_sins(rgb):
        rgb_scal = rgb/255

        max = np.max(rgb_scal)

        R = rgb_scal[0]
        G = rgb_scal[1]
        B = rgb_scal[2]

        R_1 = (50**R / 50**max)*np.sin(A_freq   * sin_scal)
        G_1 = (50**G / 50**max)*np.sin(Csh_freq * sin_scal)
        B_1 = (50**B / 50**max)*np.sin(E_freq   * sin_scal)
        return R_1 + G_1 + B_1
    
    LOC_WHITE = generate_sins(np.array(np.array([255,255,255])))

    out = np.hstack([generate_sins(rgb), np.max(LOC_WHITE)])
    
    # normalize to 16-bit range
    out*= 32767 #/ np.max(np.abs(LOC_WHITE))
    return out



## III) Examples
Running the following code will generate a series of example colors followed by each of their corresponding audios. The first 6 colors will be the same every time you run the cell, but the last 5 (in accordance with the `num_extra` variable) are randomly generated. Listen carefully to each color several times. This is your chance to get a feel for how the sonification works. Also, feel free to rerun the cell to refresh the last 5 colors by simply scrolling back up to it, clicking on it, and pressing `shift` + `return` as usual. Run the next cell.

In [ ]:
def generate_random_color():
    # Generates an np array with three entries, 
    # each ranging from 0 to 255, representing a color
    return np.random.randint(256, size=3)

def plot_colored_square(rgb):
    # Takes an np array RGB, representing a color, 
    # and plots a square corresponding to that color
    col = colors.to_hex(rgb/255)
    x,y = [-1,1,1,-1],[-1,-1,1,1]
    plt.figure(figsize=(3,3))
    plt.axis('equal')
    plt.fill(x, y, col)
    plt.show()

def generate_examples(color_list):
    #Takes a list of colors, COLOR_LIST and runs PLOT_COLORED_SQUARE for each color, 
    # and creates an audio player for the sound corresponding to that color
    for c in color_list:
        plot_colored_square(c)
        display(Audio(data = generate_sound(c), rate = sample_rate))
    
# a list of example colors RED, GREEN, BLUE, YELLOW, CYAN, MAGENTA, respectively.
color_list = [np.array([255,0,0]), np.array([0,255,0]), np.array([0,0,255]),
              np.array([200,200,0]), np.array([0,200,200]), np.array([200,0,200])]

# adds NUM_EXTRA random colors to COLOR_LIST
i = 0
num_extra = 5
while i < num_extra:
    c = generate_random_color()
    if np.min(np.linalg.norm(c - color_list, axis = 1)) < 130:
        continue
    else:
        color_list.append(c)
        i += 1

generate_examples(color_list)

## IV) Quiz 1
Up next is the first quiz question. Here, you have the chance to test your skills at interpreting the sonification. Ten audios, each with a corresponding color picker underneath will be displayed when you run the next cell. Listen to the audio, then pick the color you think it represents in the color picker. Clicking `Confirm` will lock in your answer. It is not necessary to click `Confirm`, but once you do you can no longer edit your answer. Clicking the button prevents you from going back and changing your answers after you have completed the quiz. This doesn't have a huge effect, but it could make interpreting your results more confusing. The cell after this next one compiles your results and should print out a list of numbers corresponding to your distance from the true color on each guess. Results can range from 0, meaning you guessed the color exactly to 441, meaning you were as far off as possible, eg. picking white when the correct answer was black. Distances below 100 are impressive. If you want to start the quiz over for whatever reason, return to this cell, and run the cells again as if it as if it was your first time doing the quiz. When you're ready, run the next cell, and the one after.

In [ ]:
def generate_n_pickers(n):
    #returns np array of N color pickers
    rv = np.array([])
    for _ in range(n):
        cp = widgets.ColorPicker(
        concise=True,
        description='Pick a color',
        value='white',
        disabled=False
        )
        rv = np.append(rv, cp)
    return rv

def generate_n_buttons(n):
    #returns np array of N confirmation buttons
    rv = np.array([])
    for i in range(n):
        b = widgets.Button(
        value=False,
        description='Confirm',
        disabled=False,
        tooltip=str(i),
        icon='check'
        )
        rv = np.append(rv, b)
    return rv

def generate_n_colors(n):
    #returns np array of N random colors
    rv = np.array([generate_random_color()])
    for _ in range(1,n):
        rv = np.append(rv, [generate_random_color()], axis = 0)
    return rv

def generate_n_sounds(n):
    #returns np array of N sounds from N randomly generated colors, as well as this np array of colors
    rv = np.array([])
    col = generate_n_colors(n)
    for c in col:
        a = Audio(data = generate_sound(c), rate = sample_rate)
        rv = np.append(rv, a)
    return rv, col

N = 10
pickers = generate_n_pickers(N)
buttons = generate_n_buttons(N)
sounds, targets = generate_n_sounds(N)

for i in range(N):
    display(sounds[i], pickers[i], buttons[i])

def on_button_clicked(b):
    #create individual on_button_clicked functions for each button, tying it to corresponding cp
    i = int(b.tooltip)
    pickers[i].disabled = True
    b.disabled = True

for b in buttons:
    b.on_click(on_button_clicked)

In [ ]:
guesses = np.array([np.array(colors.to_rgb(pickers[0].value)) * 255])
for i in range(1,N):
    c = np.array(colors.to_rgb(pickers[i].value)) * 255
    guesses = np.append(guesses, [c], axis = 0)

differences = guesses - targets
distances = np.linalg.norm(differences, axis = 1)
distances

The next cell will plot your results in 3 dimensions, with the x-axis corresponding to the red, the y-axis to the green, and the z-axis to the blue values of a color. Vectors corresponding to each axis's color will be plotted along each of the 3 axes. Each guess will have a number plotted next to it, and a black outline. It will also have a vector pointing to the correct answer, which will be outlined in red. Running this next cell, as well as the two that follow it will show you the 3-D plot from 3 different angles (each with a different axis as the vertical axis). Here, you will be able to see how far off your guesses were graphically. The plots tend to be cluttered, so while a guess may not be clear on one plot, it probably will be easier to discern on one or two of the others.

In [ ]:
#%matplotlib widget

fig = plt.figure(figsize=(5,5))
plt.subplot(121)
ax = Axes3D(fig)

ax.set_xlim(0,255)
ax.set_ylim(0,255)
ax.set_zlim(0,255)

#Plot red, green, and blue axes as vectors.
ax.quiver(0,0,0,255,0,0, color = 'r')
ax.quiver(0,0,0,0,255,0, color = 'g')
ax.quiver(0,0,0,0,0,255, color = 'b')

for i in range(N):
    g = guesses[i]
    t = targets[i]
    cg = colors.to_hex(g/255)
    ct = colors.to_hex(t/255)
    ax.scatter(g[0], g[1], g[2], c = cg, edgecolors = 'black', s=75, linewidth = 1)
    ax.text(g[0], g[1], g[2], '%s' % str(i), color = 'k', size = 20)
    ax.scatter(t[0], t[1], t[2], c = ct, edgecolors = 'red', s=75, linewidth = 1)
    ax.quiver(g[0], g[1], g[2], t[0] - g[0], t[1] - g[1], t[2] - g[2])
ax.view_init(40,-45,'x')

In [ ]:
ax.view_init(40,-45,'y')
ax.figure

In [ ]:
ax.view_init(40,-45,'z')
ax.figure

This next cell will simulate a large amount of random guesses, and then output your guesses' percentile (a number between 0 and 100) among those random guesses. Results under the 5th percentile indicate that your guesses were more accurate than we would reasonably expect them to be if left up to random chance, and results over the 95th percentile indicate that you are inaccurate beyond what could be attributed to random chance. Note that the following cell could take a few seconds to run, as it involves executing 10,000 random trials (see the `trials` variable).

In [ ]:
trials = 10000
rand_dists = np.array([[]])

for i in range(trials):
    sample = np.array([np.random.randint(255, size = 3)])
    for j in range(N-1):
        sample = np.append(sample, [np.random.randint(255, size = 3)], axis = 0)
    rand_dists = np.append( rand_dists, np.linalg.norm(sample - targets))

guess_dist = np.linalg.norm(distances)


def find_pct(target, data):
    if target > np.max(data):
        return 100
    if target < np.min(data):
        return 0


    diff = 100
    p = 50
    step = 50

    while np.abs(diff) > 0.1:
        diff = target - np.percentile(data, p)
        if diff > 0:
            p += step
            step /= 2
        else:
            p -= step
            step /= 2
    return p

find_pct(guess_dist, rand_dists)

## V) Quiz 2
This next quiz tests your ability to tell the distance between two colors. Ten pairs of audio samples, each followed by a slider will pop up when you run the next cell. On each slider, input what you think the distance between the two sounds you hear is. The slider goes between 0 and 441, as 0 is the minimum distance, ie. the colors are exactly the same, and 441 is the maximum possible distance, ie. one color is black, the other is white, or one color is cyan, the other is red. Note that extreme values are much less likely because the colors, when plotted in 3-D space exist inside a cube, and higher distances force possible combinations of colors into opposite corners of the cube, where space is more limited. The cell after the next will compile your results and print a list of the difference between your guess and the acutal distance between the two sounds played for each guess, respectively. If you would like to retake this quiz, return to this cell and rerun the quiz as if it was your first time. When you're ready to start the quiz, run the next cell.

In [ ]:
def generate_n_sliders(N):
    rv = np.array([])
    for _ in range(N):
        s = widgets.FloatSlider(
            value=0,
            min=0,
            max=np.linalg.norm([255,255,255]),
            step=0.1,
            description='Distance:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f')
        rv = np.append(rv, s)
    return rv

sounds, targets = generate_n_sounds(2*N)
sliders = generate_n_sliders(N)

for i in range(N):
    display(sounds[2*i], sounds[2*i+1], sliders[i])


In [ ]:
dist_diff = np.array([])
target_dists = np.array([])
guesses = np.array([])
rand_guesses = np.random.randint(np.linalg.norm([255,255,255]), size = N)
for i in range(N):
    target_dists = np.append(target_dists, np.linalg.norm((targets[2*i+1] - targets[2*i])))
    d = np.abs(target_dists[i] - sliders[i].value)
    guesses = np.append(guesses, sliders[i].value)
    dist_diff = np.append(dist_diff, d)

np.abs(guesses - target_dists)

This next cell plots your guesses (black) against a series of 10 random guesses (blue), while the true distances are plotted in red. The horizontal axis corresponds to the quiz question number, so the data for your 5th guess are the points above 5 on the horizontal axis.

In [ ]:
x = np.arange(1,N+1)
plt.scatter(x, rand_guesses)
plt.scatter(x,target_dists, color = 'r')
plt.scatter(x,guesses, color = 'black')

This last cell generates a large amount of random guesses, then outputs the percentile of your guesses among the random ones. As before results less than the 5th percentile indicate that your accuracy is due to more than just random chance.

In [ ]:
rand_dists = np.array([])
for i in range(trials):
    sample = np.random.randint(np.linalg.norm([255,255,255]), size = N)
    dist = np.linalg.norm(sample - target_dists)
    rand_dists = np.append(rand_dists, dist)

sd = np.std(rand_dists)
mean = np.mean(rand_dists)

guess_dist = np.linalg.norm(guesses - target_dists)

find_pct(guess_dist, rand_dists)

## VI) Summary
My first step in this project was determining in what way to sonify color. I settled on translating the RGB values to three different tones, as I saw this as a reasonably direct pathway, and the easiest option to measure individuals' success with. Other options included representing named colors as chords, but this proved complicated to implement, and very impractical to measure accuracy on, as the only way to determine colors from sounds is to memorize which colors correspond to which chords. Either one has this information memorzied or one does not, but there is less smooth of a gradient between knowing and not knowing. A goal of my project at the beginning was to send the quiz to several people and collect and display their results. This proved to be very complicated and difficult. Setting up the sonification took a couple of hours of work per day spanned over two or three days, and efforts to automatically collect respondents' data took just as long. If I want to achieve this functionality, I probably have to set up a website myself, and do a great deal of background work with servers and database systems, which I have no experience with. As a result, I gave up on trying to implement this functionality and settled on comparing respondents' results to a series of random guesses. I feel like this still gives users some benchmark of where they stand, and an idea of how much they can improve, but if I come back to this project I want to try to add in true survey functionality.